In [27]:
import os
import json
import pickle
import numpy as np
import cv2
from pathlib import Path
import pandas as pd
import shutil
from keras.models import load_model
from tensorflow.keras.utils import img_to_array, load_img
from PIL import ImageFile


In [28]:
BASE_MODEL_PATH = "/.model"
TEST_DIR = "csv_files/test.csv"
PREDICT_DIR = "./pred_dir"
PICKLE_DIR = "./pickle_files"
JSON_DIR = "./json_files"

In [29]:
if not os.path.exists(JSON_DIR):
    os.makedirs(JSON_DIR)


In [30]:
BEST_MODEL = "D:\DistractedDriver\model\cnn_batchwise_best_model.keras"
model = load_model(BEST_MODEL)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 128)  │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 512)    │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │    50,176,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,742,734 (388.12 MB)

 Trainable params: 50,871,366 (194.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 50,871,368 (194.06 MB)

In [31]:
data_test = pd.read_csv(Path(TEST_DIR))
#testing on the only 10000 images as loading the all test images requires ram>8gb
data_test = data_test[:10000] 
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   FileName   10000 non-null  object
 1   ClassName  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [32]:
with open("D:\DistractedDriver\pickle_files\labels_list_cnn_batchwise.pkl","rb") as handle:
    labels_id = pickle.load(handle)
print(labels_id)

{'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


In [40]:
from tqdm import tqdm
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True  
test_tensors = paths_to_tensor(data_test.iloc[:,0]).astype('float32') / 255.0

100%|██████████| 10000/10000 [00:49<00:00, 201.31it/s]


In [41]:
#The 4 lines below are only for shallow (deep - 64, nonbatch-224)
# from tensorflow.keras.applications import VGG16  
# feature_extractor = VGG16(include_top=False, input_shape=(64, 64,3), weights='imagenet')
# test_features = feature_extractor.predict(test_tensors)
# ypred_test = model.predict(test_features)
ypred_test = model.predict(test_tensors,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels)

for i in range(data_test.shape[0]):
    data_test.iloc[i,1] = id_labels[ypred_class[i]]

313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 267ms/step
{0: 'c0', 1: 'c1', 2: 'c2', 3: 'c3', 4: 'c4', 5: 'c5', 6: 'c6', 7: 'c7', 8: 'c8', 9: 'c9'}


In [42]:
#to create a human readable and understandable class_name 
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"


with open(os.path.join(JSON_DIR,'class_name_map.json'),'w') as secret_input:
    json.dump(class_name,secret_input,indent=4,sort_keys=True)

In [43]:
# creating the prediction results for the image classification and shifting the predicted images to another folder
#with renamed filename having the class name predicted for that image using model
with open(os.path.join(JSON_DIR,'class_name_map.json')) as secret_input:
    info = json.load(secret_input)
PREDICT_DIR="./pred_dir/cnn_batchwise"
if not os.path.exists(PREDICT_DIR):
    os.makedirs(PREDICT_DIR)

for i in range(data_test.shape[0]):
    new_name = data_test.iloc[i,0].split("/")[-1].split(".")[0]+"_"+info[data_test.iloc[i,1]]+".jpg"
    shutil.copy(data_test.iloc[i,0],os.path.join(PREDICT_DIR,new_name))
    
#saving the model predicted results into a csv file
data_test.to_csv(os.path.join("./csv_files","cnn_batchwise.csv"),index=False)

In [39]:
test_img = "D:/DistractedDriver/dataset/imgs/test/img_4.jpg"


img = load_img(test_img, target_size=(224,224))
x = img_to_array(img).astype("float32") / 255.0
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
print("Raw prediction:", pred)
print("Argmax:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Raw prediction: [[0.03006753 0.00157617 0.005964   0.04023506 0.0394276  0.7501219
  0.00189337 0.061239   0.03601637 0.03345898]]
Argmax: 5
